<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных-из-файла" data-toc-modified-id="Изучение-данных-из-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных из файла</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Расчёты-и-добавление-результатов-в-таблицу" data-toc-modified-id="Расчёты-и-добавление-результатов-в-таблицу-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Расчёты и добавление результатов в таблицу</a></span></li><li><span><a href="#Исследовательский-анализ-данных" data-toc-modified-id="Исследовательский-анализ-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Исследовательский анализ данных</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

## Изучение данных из файла

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
print(data.shape) #оцениваем датафрейм
display(data.info()) #оцениваем тип данных
display(data.head(5)) #смотрим первые 5 строк

(23699, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
total_images            23699 non-null int64
last_price              23699 non-null float64
total_area              23699 non-null float64
first_day_exposition    23699 non-null object
rooms                   23699 non-null int64
ceiling_height          14504 non-null float64
floors_total            23613 non-null float64
living_area             21796 non-null float64
floor                   23699 non-null int64
is_apartment            2775 non-null object
studio                  23699 non-null bool
open_plan               23699 non-null bool
kitchen_area            21421 non-null float64
balcony                 12180 non-null float64
locality_name           23650 non-null object
airports_nearest        18157 non-null float64
cityCenters_nearest     18180 non-null float64
parks_around3000        18181 non-null float64
parks_nearest           8079 non-null float64
pon

None

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,...,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,...,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,...,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,...,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


### Вывод

***Наши данные***
У нас 23 699 строк, часть данных целочисленные, часть дробные, есть булевые значения и текстовые. Есть пропуски, надо решить что с ними делать. Есть даты - надо будет переводить в формат даты. 

## Предобработка данных

In [13]:
data['balcony']= data['balcony'].fillna(0)
print(data.info())
print(data['is_apartment'].unique()) #здесь должны быть булевы значения, но вместо этого есть пропуски. Мне не понятно пока, как это можно заменить. 
data['last_price'] = data['last_price'].astype('int') #заменим вещественный тип на целочисленный в ценах, т.к. цены в рублях и копейки не имеют значения при таких суммах
data['floors_total'] = data['floors_total'].fillna(0).astype('int') #этажность имеет значение. При этом пропущенных значений в этажности менее 1%, так что не страшно, если мы заменим на 0 и на целые значения 
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S') #переводим значения в first_day_exposition  в формат даты
data = data.dropna(subset=['locality_name']) #в будущем нам понадобится анализировать населенные пункты, поэтому строки без указания населенного пункта удалим, их очень мало
data.loc[data['kitchen_area'].isna(), 'kitchen_area'] = data['kitchen_area'].median() #заполнили пропуски в данных по площади кухни медианными значениями
for floor in data['floors_total'].unique():
    median = data.loc[data['floors_total'] == floor, 'ceiling_height'].median()
    data.loc[(data['ceiling_height'].isna()) & (data['floors_total'] == floor), 'ceiling_height'] = median #заменим отсутствующие значения в ceiling_height на медиану по сгруппированным значениям этажности дома. Только я не понимаю, почему получилось 23 649 значений?...
print(data.info())
display(data.head(10))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 23650 entries, 0 to 23698
Data columns (total 26 columns):
total_images            23650 non-null int64
last_price              23650 non-null int64
total_area              23650 non-null float64
first_day_exposition    23650 non-null datetime64[ns]
rooms                   23650 non-null int64
ceiling_height          23649 non-null float64
floors_total            23650 non-null int64
living_area             21752 non-null float64
floor                   23650 non-null int64
is_apartment            2760 non-null object
studio                  23650 non-null bool
open_plan               23650 non-null bool
kitchen_area            23650 non-null float64
balcony                 23650 non-null float64
locality_name           23650 non-null object
airports_nearest        18116 non-null float64
cityCenters_nearest     18139 non-null float64
parks_around3000        18140 non-null float64
parks_nearest           8064 non-null float64
ponds_aroun

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition,price_m,month,week_day,year
0,20,13000000,108.00,2019-03-07,3,2.70,16,51.00,8,NaN,...,16028.0,1.0,482.0,2.0,755.0,NaN,120370,3,3,2019
1,7,3350000,40.40,2018-12-04,1,2.75,11,18.60,1,NaN,...,18603.0,0.0,NaN,0.0,NaN,81.0,82920,12,1,2018
2,10,5196000,56.00,2015-08-20,2,2.55,5,34.30,4,NaN,...,13933.0,1.0,90.0,2.0,574.0,558.0,92785,8,3,2015
3,0,64900000,159.00,2015-07-24,3,2.60,14,NaN,9,NaN,...,6800.0,2.0,84.0,3.0,234.0,424.0,408176,7,4,2015
4,2,10000000,100.00,2018-06-19,2,3.03,14,32.00,13,NaN,...,8098.0,2.0,112.0,1.0,48.0,121.0,100000,6,1,2018
5,10,2890000,30.40,2018-09-10,1,2.65,12,14.40,5,NaN,...,NaN,NaN,NaN,NaN,NaN,55.0,95065,9,0,2018
6,6,3700000,37.30,2017-11-02,1,2.66,26,10.60,6,NaN,...,19143.0,0.0,NaN,0.0,NaN,155.0,99195,11,3,2017
7,5,7915000,71.60,2019-04-18,2,2.75,24,NaN,22,NaN,...,11634.0,0.0,NaN,0.0,NaN,NaN,110544,4,3,2019
8,20,2900000,33.16,2018-05-23,1,2.70,27,15.43,26,NaN,...,NaN,NaN,NaN,NaN,NaN,189.0,87454,5,2,2018
9,18,5400000,61.00,2017-02-26,3,2.50,9,43.60,7,NaN,...,15008.0,0.0,NaN,0.0,NaN,289.0,88524,2,6,2017


## Расчёты и добавление результатов в таблицу

In [3]:
data['price_m'] = (data['last_price']/data['total_area']).astype('int') #вычислим цену квадратного метра и сразу переведем в целочисленные значения
data['month'] = pd.DatetimeIndex(data['first_day_exposition']).month #добавим колонку "Месяц" в таблицу - месяц публикации объявления
data['week_day'] = pd.DatetimeIndex(data['first_day_exposition']).weekday #добавим колонку "День недели" в таблицу - день недели публикации объявления
data['year'] = pd.DatetimeIndex(data['first_day_exposition']).year #добавим колонку "Год" в таблицу - год публикации объявления
display(data[['month', 'first_day_exposition', 'week_day', 'year']].head(5)) #проверим, что сделали всё верно
print(data['floor'].unique())

        

    


,month,first_day_exposition,week_day,year
0,3,2019-03-07,3,2019
1,12,2018-12-04,1,2018
2,8,2015-08-20,3,2015
3,7,2015-07-24,4,2015
4,6,2018-06-19,1,2018


[ 8  1  4  9 13  5  6 22 26  7  3  2 11 15 18 10 19 16 20 27 25 17 14 12
 21 28 24 23 30 29 32 33 31]


## Исследовательский анализ данных

## Общий вывод

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл
- [ ]  файлы изучены (выведены первые строки, метод info())
- [ ]  определены пропущенные значения
- [ ]  заполнены пропущенные значения
- [ ]  есть пояснение, какие пропущенные значения обнаружены
- [ ]  изменены типы данных
- [ ]  есть пояснение, в каких столбцах изменены типы и почему
- [x]  посчитано и добавлено в таблицу: цена квадратного метра
- [x]  посчитано и добавлено в таблицу: день недели, месяц и год публикации объявления
- [ ]  посчитано и добавлено в таблицу: этаж квартиры; варианты — первый, последний, другой
- [ ]  посчитано и добавлено в таблицу: соотношение жилой и общей площади, а также отношение площади кухни к общей
- [ ]  изучены следующие параметры: площадь, цена, число комнат, высота потолков
- [ ]  построены гистограммы для каждого параметра
- [ ]  выполнено задание: "Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Опишите, сколько обычно занимает продажа. Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?"
- [ ]  выполнено задание: "Уберите редкие и выбивающиеся значения. Опишите, какие особенности обнаружили."
- [ ]  выполнено задание: "Какие факторы больше всего влияют на стоимость квартиры? Изучите, зависит ли цена от квадратного метра, числа комнат, этажа (первого или последнего), удалённости от центра. Также изучите зависимость от даты размещения: дня недели, месяца и года. "Выберите 10 населённых пунктов с наибольшим числом объявлений. Посчитайте среднюю цену квадратного метра в этих населённых пунктах. Выделите населённые пункты с самой высокой и низкой стоимостью жилья. Эти данные можно найти по имени в столбце '*locality_name'*. "
- [ ]  выполнено задание: "Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра. Выделите квартиры в Санкт-Петербурге (*'locality_name'*). Ваша задача — выяснить, какая область входит в центр. Создайте столбец с расстоянием до центра в километрах: округлите до целых значений. После этого посчитайте среднюю цену для каждого километра. Постройте график: он должен показывать, как цена зависит от удалённости от центра. Определите границу, где график сильно меняется — это и будет центральная зона. "
- [ ]  выполнено задание: "Выделите сегмент квартир в центре. Проанализируйте эту территорию и изучите следующие параметры: площадь, цена, число комнат, высота потолков. Также выделите факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления). Сделайте выводы. Отличаются ли они от общих выводов по всему городу?"
- [ ]  в каждом этапе есть выводы
- [ ]  есть общий вывод